In [1]:
import pandas as pd
import sys
sys.path.append("../src")
import data_preparation as Clean
import Neural_Networks

In [2]:
train = pd.read_csv("../data/train.csv")

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_9515/2794332508.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/train.csv")


In [3]:
model = Neural_Networks.Model_NN(train, test)

(7571649, 37)

In [4]:
model.fit(num_epochs=10)

Entraînement débuté


/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([128, 9])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Users/axel/Documents/École/Université/M2/M2_S1/Data_Mining/ChallengeKaggle/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([90, 9])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (128) must match the size of tensor b (90) at non-singleton dimension 0

In [3]:
dataprep = Clean.DataPreparation(train, test, neural_networks=True, target = 'Ewltp_(g/km)')
X_train, X_val, y_train, y_val = dataprep.prepare_data()

Valeurs manquantes du train supprimées ✅
Valeurs manquantes du test supprimées ✅
Variables renommées ✅
Valeurs manquantes numériques imputées ✅
Valeurs manquantes catégorielles imputées ✅


In [7]:
from tqdm import tqdm
############################################# Imports #################################################

## Neural Networks' Packages
import torch
from torch import nn
from sklearn.metrics import mean_absolute_error
## Data Preparation
import data_preparation as Clean
import time

############################################# Model #################################################

class Model_NN1(nn.Module):
    def __init__(self, train, test):
        super().__init__()
        self.train_data = train
        self.test_data = test

        self.dataprep =  dataprep #Clean.DataPreparation(self.train, self.test, neural_networks=True, target = 'Ewltp_(g/km)')
        self.X_train, self.y_train, self.X_val, self.y_val =  X_train, y_train, X_val, y_val#self.dataprep.prepare_data()

        self.input_dim = self.X_train.shape[1]

        self.h1_dim = 128
        self.h2_dim = 256
        self.h3_dim = 256
        self.h4_dim = 128
        self.output = 1

        #layers
        self.tab_2hid1 = nn.Linear(self.input_dim, self.h1_dim)
        self.dropout1 = nn.Dropout(0.5)  
        self.hid1_2hid2 = nn.Linear(self.h1_dim, self.h2_dim)
        self.dropout2 = nn.Dropout(0.5)
        self.hid2_2hid3 = nn.Linear(self.h2_dim, self.h3_dim)
        self.dropout3 = nn.Dropout(0.5) 
        self.hid3_2hid4 = nn.Linear(self.h3_dim, self.h4_dim)
        self.dropout4 = nn.Dropout(0.5)
        self.hid4_2output = nn.Linear(self.h4_dim, self.output)

        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Run the whole model.

        Args :
            - x (torch) : data to synthesize

        Return :
            tuple :
                - x_reconstructed : synthesized data
                - mu and sigma : distribution parameter

        """

        x = self.relu(self.tab_2hid1(x))
        x = self.dropout1(x)
        x = self.relu(self.hid1_2hid2(x))
        x = self.dropout2(x)
        x = self.relu(self.hid2_2hid3(x))
        x = self.dropout3(x)
        x = self.relu(self.hid3_2hid4(x))
        x = self.dropout4(x)
        x = self.hid4_2output(x)
        return x

    def fit(self, num_epochs=300, LR_rate=1e-3, display=False):
        """
        This function trains the model.

        Args :
            - num_epochs (int) : number of times the model will be trained.
            - LR_rate (float64) : Learning rate.
            - display (bool) : display or not the information during the training process.
        """
        print("Entraînement débuté")
        start_time = time.time()
        self.device = torch.device('cpu')

        optimizer = torch.optim.AdamW(self.parameters(), lr=LR_rate)
        MAE_loss = nn.L1Loss()

        for epoch in range(num_epochs):
            for x, y in tqdm(zip(self.dataprep.train_dataloader, self.dataprep.target_dataloader)) :
                x.to(self.device), y.to(self.device)
                output = self.forward(x)
                # loss
                loss = MAE_loss(output, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            print(f"Epoch {epoch} terminée, loss = {loss}")

        print("Entraînement terminé")
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Temps total de l'entraînement : {round(elapsed_time,2)} secondes")

    def predict(self):
        self.eval()  
        with torch.no_grad():  
            data_np = self.X_val.values if isinstance(self.X_val, pd.Series) else self.X_val.to_numpy()
            data = torch.from_numpy(data_np).float()  
            data = data.to(self.device)
            
            predictions = self.forward(data)  
            predictions = predictions.numpy()
            
            mae_loss = mean_absolute_error(predictions, self.y_val)
            print("La MAE sur l'ensemble de validation est de ", mae_loss)
        return predictions, mae_loss

In [8]:
model = Model_NN1(train, test)

In [9]:
model.fit(num_epochs=20, display=True)

Entraînement débuté


23662it [02:38, 149.09it/s]


Epoch 0 terminée, loss = 50.10544967651367


23662it [02:38, 148.90it/s]


Epoch 1 terminée, loss = 52.495140075683594


...

Epoch 7 terminée, loss = 60.50236892700195


23662it [02:40, 147.73it/s]


Epoch 8 terminée, loss = 57.906002044677734


3707it [00:25, 146.51it/s]


KeyboardInterrupt: 

In [8]:
pred, loss = model.predict()

La MAE sur l'ensemble de validation est de  11.598710280705555
